In [2]:
api_key = 'API KEY'
api_secret = 'API SECRET'
import pandas as pd
from binance.client import Client
import matplotlib.pyplot as plt
from datetime import datetime
import pandas_ta as ta


In [3]:
client = Client(api_key, api_secret)

In [4]:
ticker = 'BTCUSDT'
interval = Client.KLINE_INTERVAL_5MINUTE
#depth = '12 hours ago'

raw = client.get_historical_klines(ticker, interval, "01 Jan, 2020", "01 Nov, 2020")

In [5]:
btc_df = pd.DataFrame(raw, columns=['date', 'open', 'high', 'low', 'close', 'Volume', 'Close Time', 'Quote Asset Volume', 'Number of trades', 'Taker buy base', 'Taker buy quote asset volume', 'ignore'])
btc_df.set_index('date', inplace=True)
btc_df.index = pd.to_datetime(btc_df.index, unit='ms')
btc_df['Close Time'] = pd.to_datetime(btc_df['Close Time'], unit='ms')
btc_df['open'] = btc_df.high.astype(float)
btc_df['high'] = btc_df.high.astype(float)
btc_df['low'] = btc_df.high.astype(float)
btc_df['close'] = btc_df.high.astype(float)
btc_df['Volume'] = btc_df.high.astype(float)

btc_df = btc_df.drop(btc_df.columns[[5, 6, 7, 8, 9, 10]], axis=1)


In [6]:
sma_10 = ta.sma(btc_df["Volume"], length=10) #50 mins
sma_50 = ta.sma(btc_df["Volume"], length=50) # 4 hours

sma_close_f = ta.sma(btc_df["close"], length=2) # 25 mins
sma_close_s = ta.sma(btc_df["close"], length=4) # 25 mins

macd = ta.macd(btc_df['close'])


In [9]:
#df = pd.DataFrame()
money = 50
btc = 0
cnt = 0
for index, row in macd.iterrows():
    c = round(float(row['MACD_12_26_9']), 1)
    #s = round(float(row['MACDs_12_26_9']), 1)
    
    vol_con = float(sma_10.loc[[index]].values) - float(sma_50.loc[[index]].values)
    if money > 0 and c < 0 and vol_con >-2:
        cnt += 1
        buy_price = float(btc_df.loc[[index]]['close'].values)
        #print("Buy: ", buy_price)
        btc = (money / buy_price) - ((money / buy_price)*0.0001)
        money = 0
    if btc > 0 and (c > 10):
        sell_price = float(btc_df.loc[[index]]['close'].values)
        #print("Sell", sell_price)
        money = (btc * sell_price) - ((btc * sell_price)*0.0001)
        btc = 0
    #df = df.append({'Date': index, 'price': money}, ignore_index=True)
if btc > 0:
    print("took the losing trade at the end")
    dat = float(btc_df.tail(1)['close'].values)
    money = (btc * dat) - ((btc * dat)*0.0001)

print("money = ", money)
print("number of buys = ", cnt)


took the losing trade at the end
money =  117.45442112927478
number of buys =  686


~~~~~~~THIS IS IT~~~~~~~~~~~~